Submission for<br>
Exercise task 9<br>
of UTU course TKO_8964-3006<br>
Textual Data Analysis<br>
by Botond Ortutay<br>

---

In this exercice, we will work on relation extraction part of the [Bacteria Biotope task at BioNLP Open Shared Tasks 2019](https://aclanthology.org/D19-5719/). Although the original task in the paper is more complicated (covers more relation types), here we have simplified the dataset, so that you can focus on extracting binary relations with GPT. 

The task is a binary relation extraction, with three entity types:
 - Microorganism (e.g. bacteria)
 - Habitat (e.g. milk, cheese, etc)
 - Geographical (e.g. Europe)

and one binary relaiton type: *Lives_in* in the form of:
 - *Lives_in*(Microorganism , Habitat)
 - *Lives_in*(Microorganism , Geographical)

Therefore, prediction outcome is **Positive** if the sentence states that the Microorganism lives in the mentioned Habitat/Geographical and **Negative**, otherwise. 

For example:
 - **Salmonella** was noticed in **raw chicken**. --> Positive --> Lives_in (Salmonella, raw chicken)
 - **Salmonella** was not noticed in **cooked chicken**. --> Negative

You are provided with a Colab [notebook](https://github.com/TurkuNLP/textual-data-analysis-course/blob/main/tda_2025_Ex9_2025_Rel_withGPT.ipynb) that already covers about 80% of the work. Your task is to complete the remaining 20%. For example, code to download the data, generate candidate pairs, mark entity spans, and even prompting GPT to predict the labels for the examples are given. 

For using GPT, use the **api-key** that you were provided in **Exercise task 6**.

First, familiarize yourself with the data and the code, and then complete the following tasks.

**Task1**:

Write a code that predicts the labels for all examples in `filtered_devel_examples` (there are 91 examples in this set) by asking the GPT model. You **must** use a `for` loop that packs **5 examples at-a-time** (in each iteration of the for loop), and passes to GPT, and predicts the labels for them at once. This is to keep GPT costs low (a real situation if you're going to work for a company). Then use `sklearn.metrics.precision_recall_fscore_support` to report precision, recall, and f1-score for the `filtered_devel_examples`. Do **NOT** give any example inputs to the GPT (**no few-shot learning in this step**).

**Task2**:

Now, include the two positive and negative sentence about Salmonella and chicken that I had mentioned, as examples in your prompts.
 - "UID: UID_1000 Text = Salmonella was noticed in raw chicken. --> {'UID':'UID_1000', label = 'Positive'}
 - "UID: UID_1001 Text = Salmonella was not noticed in coocked chicken. --> {'UID':'UID_1001', label = 'Negative'}"

and repeat the experiment, and report precision, recall, and f1-score. How the results have changed?

**Task3**:

3.1 Now **randomly** select 4 examples from the **training set** (you can take 2 positives, 2 negatives, or really sample randomly) and include them in your prompt, repeat the experiment, report precision, recall, and f1-score. How the results have changed?

3.2 Repeat above experiment: Again, **randomly** sample 4 training examples and use them **instead** of the 4 previous examples. Have the results changed? **Why?**

**IMPORTANT**: In order not to get a time-out error from GPT call, you can increase the timeout, **but also make sure you sample from smaller texts**.

---

**Library imports**

In [8]:
from itertools import product
import json

---

**Code from [colab notebook](https://github.com/TurkuNLP/textual-data-analysis-course/blob/main/tda_2025_Ex9_2025_Rel_withGPT.ipynb) (provided in instructions) (small modifications):**

In [1]:
#bash scripts here
# downloading datasets
!wget -O bb4_converted_train.json https://raw.githubusercontent.com/TurkuNLP/textual-data-analysis-course/main/BB4_converted_train.json
!wget -O bb4_converted_devel.json https://raw.githubusercontent.com/TurkuNLP/textual-data-analysis-course/main/BB4_converted_devel.json



7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://raw.githubusercontent.]87Saving 'bb4_converted_train.json'
87bb4_converted_train. 100% [=============================>]   74.24K    --.-KB/s87HTTP response 200  [https://raw.githubusercontent.com/TurkuNLP/textual-data-analysis-course/main/BB4_converted_train.json]
87bb4_converted_train. 100% [=============================>]   74.24K    --.-KB/s87[Files: 1  Bytes: 74.24K [205.1]8

7[Files: 0  Bytes: 0  [0 B/s] Re]87[https://raw.githubusercontent.]87Saving 'bb4_converted_devel.json'
87bb4_converted_devel. 100% [=============================>]   41.83K    --.-KB/s87HTTP response 200  [https://raw.githubusercontent.com/TurkuNLP/textual-data-analysis-course/main/BB4_converted_devel.json]
87bb4_converted_devel. 100% [=============================>]   41.83K    --.-KB/s87[Files: 1  Bytes: 41.83K [99.13]8

**Files in:**

In [4]:
def read_json(file_name):
  with open(file_name, "rt" , encoding = 'utf-8') as f:
    data = json.load(f)
  return data

train = read_json("bb4_converted_train.json")
devel = read_json("bb4_converted_devel.json")

print("training documents:" , len(train))
print("development documents:" , len(devel))

training documents: 125
development documents: 64


**Exploring dataformat:**

In [5]:
print (train.keys())
train_doc_id = "BB-rel-F-22177851-012"
for item in train[train_doc_id].items():
  print (item)

dict_keys(['BB-rel-F-22177851-006', 'BB-rel-F-22177851-012', 'BB-rel-18845825', 'BB-rel-9526514', 'BB-rel-12970344', 'BB-rel-F-22177851-013', 'BB-rel-F-22177851-007', 'BB-rel-6143890', 'BB-rel-17237163', 'BB-rel-10738994', 'BB-rel-16990433', 'BB-rel-24678646', 'BB-rel-F-22177851-011', 'BB-rel-F-22177851-005', 'BB-rel-F-22177851-004', 'BB-rel-25634638', 'BB-rel-F-22177851-010', 'BB-rel-8607503', 'BB-rel-F-18524407-001', 'BB-rel-F-22177851-014', 'BB-rel-F-22177851-000', 'BB-rel-F-23290227-000', 'BB-rel-8347510', 'BB-rel-F-22177851-001', 'BB-rel-F-22177851-015', 'BB-rel-448557', 'BB-rel-F-18524407-000', 'BB-rel-14645268', 'BB-rel-11410343', 'BB-rel-21917915', 'BB-rel-F-22177851-003', 'BB-rel-F-22177851-017', 'BB-rel-9643457', 'BB-rel-F-22177851-002', 'BB-rel-16263187', 'BB-rel-F-25496341-009', 'BB-rel-6631408', 'BB-rel-24831788', 'BB-rel-2696427', 'BB-rel-F-25496341-008', 'BB-rel-19099664', 'BB-rel-F-23224222-009', 'BB-rel-F-23224222-008', 'BB-rel-16432479', 'BB-rel-25098305', 'BB-rel-243

As you can see, each document in the training or development set has a key, and it is a python dictionary, with "text" , "entities" , and "relations".
 - Entities have ids, starting with "T", type, and begining and end offsets in the text. **Some entities have multiple spans.**
 - Relations are also have ids, starting with "R" , and two arguments (entities). The first argument (e1), is always a Microorganism, and the second argument (e2) is either a "Habitat", or a "Geographical" location.

**because this is manually annotated dataset, only positive pairs are annotated. We can generate all candidate pairs, and those that are annotated, are Positive examples, and those which we cannot find in the relations part, are the negative examples.**

In the above example, there is only one Habitat, one Microorganism, and one (Positive) relation between them.

**But a document can have more than two entities and relations. Here is an example**

In [6]:
train_doc_id = "BB-rel-F-22177851-006"
print ("TEXT=", train[train_doc_id]['text'])
print ("Entities:")
for entity_id in train[train_doc_id]['entities']:
  print (entity_id , ":" , train[train_doc_id]['entities'][entity_id])

print ("Positive relations:")
for positive_relation_id in train[train_doc_id]['relations']:
  print(positive_relation_id, ":" , train[train_doc_id]['relations'][positive_relation_id])
print("-"*80)

TEXT= The strain S3 + and its protease-negative variant S3-, assigned to Lactococcus lactis sp. lactis, was used for cheese-making. The 11 microorganisms that composed the model community included seven bacteria — Lactobacillus casei FH1, Arthrobacter arilaitensis Re117, Corynebacterium casei Mu120, C. flavescens Mu128, C. variabile Mu129, Staphylococcus xylosus Com1 and S. equorum Mu2 — and four yeasts — D. hansenii DH68, G. candidum GC129, Y. lipolytica CI35 and K. lactis
 KL65. This composition is representative of the diversity of smear soft
 cheese. The microorganisms were originally isolated from Munster except
 for Re117 (Reblochon), Com1 (unknown) and FH1 (St. Nectaire). The 
Gram-negative bacteria studied, P. celer 91 and H. alvei
 2 920, isolated from Livarot and Munster cheese, respectively, were 
chosen from a large cheese bacteria collection for their low level of 
sanitary risk (sensitivity to 24 antibiotics, no production of biogenic 
amines in synthetic media and no gro

Now, let's make a function that can tag entity spans, based on entity types:

In [10]:
def mark_entities_with_entity_tags(text, entities):
    """
    Annotate text with XML-like tags inserted directly into the text.

    For each entity we assume the structure:
      { "type": , "offsets": [[start, end]], "text": <...> }

    Overlapping annotations will be handled by inserting tags at the boundaries.
    (If spans cross rather than nest, the output may have crossing tags; in many
    applications you would split such spans to get a well-formed nesting.)

    Returns:
      A new string with opening and closing tags inserted.
      For example: experimental smear soft cheese
    """
    events = []
    # For each entity (each value in the dictionary) add a start event and an end event.
    for ent in entities.values():
        tag = ent["type"]
        for start, end in ent["offsets"]:
            # Save event as (position, event_type, auxiliary, tag)
            # For a start event, we record the end position as auxiliary (so we can sort descending by end)
            # For an end event, we record the start position (so we can sort ascending by start).
            events.append((start, "start", end, tag))
            events.append((end, "end", start, tag))

    # Define a sort key so that:
    #   - Primary: sort by position.
    #   - At the same position, "end" events come before "start" events.
    #   - For start events at the same position, those with later (higher) end come first (so that outer tags open first).
    #   - For end events at the same position, those with earlier start come first.
    def event_sort_key(e):
        pos, etype, aux, tag = e
        if etype == "end":
            return (pos, 0, aux)  # lower auxiliary value first
        else:  # "start"
            return (pos, 1, -aux)  # higher aux (later end) first

    events.sort(key=event_sort_key)

    # Now, we iterate over the sorted events and insert tag strings at the appropriate positions.
    # We'll build a list of (position, string_to_insert) items.
    inserts = []
    for pos, etype, aux, tag in events:
        if etype == "end":
            inserts.append((pos, f"</{tag}>"))
        else:
            inserts.append((pos, f"<{tag}>"))

    # Because multiple inserts may occur at the same position, group them.
    # We already sorted events so that the insertions are in the correct order.
    # Now, build the annotated text by iterating through the text and inserting tags at the right offsets.
    result = []
    last_index = 0
    for pos, insert_text in inserts:
        # Append the text between the last index and current position.
        if pos > last_index:
            result.append(text[last_index:pos])
        # Append the tag.
        result.append(insert_text)
        last_index = pos
    # Append any remaining text.
    result.append(text[last_index:])
    return "".join(result)

train_doc_id = "BB-rel-607884"
for item in train[train_doc_id].items():
  print (item)

print ("-"*80)
text = train[train_doc_id]['text']
entities = train[train_doc_id]['entities']
print ("ORIGINAL TEXT :  ", text)
print ("ENCODED TEXT :   ", mark_entities_with_entity_tags(text, entities))
print ("-"*80)

('text', '[The infections from "Serratia" in the Hospital S. Camillo De Lellis of Roma (Italy) (author\'s transl)].\n\n')
('entities', {'T2': {'type': 'Microorganism', 'offsets': [[22, 30]], 'text': 'Serratia'}, 'T3': {'type': 'Habitat', 'offsets': [[39, 68]], 'text': 'Hospital S. Camillo De Lellis'}, 'T4': {'type': 'Geographical', 'offsets': [[72, 76]], 'text': 'Roma'}, 'T5': {'type': 'Geographical', 'offsets': [[78, 83]], 'text': 'Italy'}})
('relations', {'R1': {'type': 'Lives_In', 'e1': 'T2', 'e2': 'T5', 'e1_type': 'Microorganism', 'e2_type': 'Location'}, 'R2': {'type': 'Lives_In', 'e1': 'T2', 'e2': 'T3', 'e1_type': 'Microorganism', 'e2_type': 'Location'}, 'R3': {'type': 'Lives_In', 'e1': 'T2', 'e2': 'T4', 'e1_type': 'Microorganism', 'e2_type': 'Location'}})
--------------------------------------------------------------------------------
ORIGINAL TEXT :   [The infections from "Serratia" in the Hospital S. Camillo De Lellis of Roma (Italy) (author's transl)].


ENCODED TEXT :    [The

**There are two possibilities now:**
 1. Marking full texts with all entities, and asking GPT to extract all possible relations Lives_in(Microorganism, Habitat/Geographical), and hope for the best. But then we will need to process the output for an accurate evaluation, **which can get tricky.**
 2. Generative all possible positive and negative pairs, assign them unique relation id, then feeding each example separately to GPT (or pack them together), and asking for predicting the label to be positive or negative.

Here is the second approach:

In [11]:
def generate_and_mark_all_candidate_pairs(_dataset):
  unique_relation_id_index = 0
  results = []
  for document_id in _dataset:
    text = _dataset[document_id]['text']
    entities = _dataset[document_id]['entities']
    ent_microorganisms = [e_id for e_id in entities if entities[e_id]['type'] == "Microorganism"]
    ent_locations = [e_id for e_id in entities if entities[e_id]['type'] in ["Geographical", "Habitat"]]
    # generate all possible pairs
    candidate_pairs = list(product(ent_microorganisms, ent_locations))

    # look at relations to get information about all positive pairs
    relations = _dataset[document_id]['relations']
    positive_pairs = (relations.items())
    positive_pairs = set([(b['e1'], b['e2']) for (a, b) in relations.items()])

    for (a, b) in candidate_pairs:
      this_example_entities = dict()
      this_example_entities[a] = entities[a]
      this_example_entities[b] = entities[b]
      if (a, b) in positive_pairs:
        this_example_label = "Positive"
      else:
        this_example_label = "Negative"

      ml_example = {"UID" : "UID_" + str(unique_relation_id_index),
                    "doc_id": document_id,
                    "e1" : a ,
                    "e2" : b ,
                    "text": text,
                    "marked_text": mark_entities_with_entity_tags(text, this_example_entities),
                    "label": this_example_label}
      results.append(ml_example)
      unique_relation_id_index += 1
  return results

train_examples = generate_and_mark_all_candidate_pairs(train)
devel_examples = generate_and_mark_all_candidate_pairs(devel)

for example in train_examples:
  if example['doc_id'] == "BB-rel-607884":
    print(example)
print("-"*80)

{'UID': 'UID_3158', 'doc_id': 'BB-rel-607884', 'e1': 'T2', 'e2': 'T3', 'text': '[The infections from "Serratia" in the Hospital S. Camillo De Lellis of Roma (Italy) (author\'s transl)].\n\n', 'marked_text': '[The infections from "<Microorganism>Serratia</Microorganism>" in the <Habitat>Hospital S. Camillo De Lellis</Habitat> of Roma (Italy) (author\'s transl)].\n\n', 'label': 'Positive'}
{'UID': 'UID_3159', 'doc_id': 'BB-rel-607884', 'e1': 'T2', 'e2': 'T4', 'text': '[The infections from "Serratia" in the Hospital S. Camillo De Lellis of Roma (Italy) (author\'s transl)].\n\n', 'marked_text': '[The infections from "<Microorganism>Serratia</Microorganism>" in the Hospital S. Camillo De Lellis of <Geographical>Roma</Geographical> (Italy) (author\'s transl)].\n\n', 'label': 'Positive'}
{'UID': 'UID_3160', 'doc_id': 'BB-rel-607884', 'e1': 'T2', 'e2': 'T5', 'text': '[The infections from "Serratia" in the Hospital S. Camillo De Lellis of Roma (Italy) (author\'s transl)].\n\n', 'marked_text': '

This is another document with id = BB-rel-F-22177851-000

In [12]:
for item in train["BB-rel-F-22177851-000"].items():
  print (item)
print ("-"*40)
for example in train_examples:
  if example['doc_id'] == "BB-rel-F-22177851-000":
    print(example)

('text', 'Ecological and aromatic impact of two Gram-negative bacteria (Psychrobacter celer and Hafnia alvei) inoculated as part of the whole microbial community of an experimental smear soft cheese\n')
('entities', {'T2': {'type': 'Microorganism', 'offsets': [[62, 81]], 'text': 'Psychrobacter celer'}, 'T3': {'type': 'Microorganism', 'offsets': [[86, 98]], 'text': 'Hafnia alvei'}, 'T4': {'type': 'Habitat', 'offsets': [[132, 187]], 'text': 'microbial community of an experimental smear soft chees'}, 'T5': {'type': 'Habitat', 'offsets': [[158, 188]], 'text': 'experimental smear soft cheese'}})
('relations', {'R2': {'type': 'Lives_In', 'e1': 'T2', 'e2': 'T5', 'e1_type': 'Microorganism', 'e2_type': 'Location'}, 'R4': {'type': 'Lives_In', 'e1': 'T3', 'e2': 'T5', 'e1_type': 'Microorganism', 'e2_type': 'Location'}})
----------------------------------------
{'UID': 'UID_856', 'doc_id': 'BB-rel-F-22177851-000', 'e1': 'T2', 'e2': 'T4', 'text': 'Ecological and aromatic impact of two Gram-negative 

Now let's look how many positive and negative examples are in the devel set: